In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../../../rings_varying_radius"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 128
NUM_HIDDEN_LOCAL = 128
NUM_STATS = 128

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE = 1e-3

CUDA = torch.cuda.is_available()
PATH = 'AG-rad-20k-beta'
DEVICE = torch.device('cuda:1')

noise_sigma = torch.ones(1) * 0.05
if CUDA:
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (noise_sigma, N, K, D, MCMC_SIZE)

In [3]:
from local_enc_eta import *
from global_oneshot_eta_v3 import *
from global_enc_eta_v3 import *
# initialization
oneshot_eta = Oneshot_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE)
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE)
if CUDA:
    oneshot_eta.cuda().to(DEVICE)
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)

models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep_rad_v3 import *
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
train(models, EUBO, optimizer, Data, Model_Params, Train_Params)

epoch: 0\500 (232s),  symKL_DB_eta: 2344.378,  symKL_DB_z: 8078.599,  gap: 10367.488,  loss: -2267.807,  ess: 4.159
epoch: 1\500 (232s),  symKL_DB_eta: 1703.869,  symKL_DB_z: 986.943,  gap: 2652.383,  loss: -2502.197,  ess: 4.745
epoch: 2\500 (231s),  symKL_DB_eta: 1591.801,  symKL_DB_z: 997.914,  gap: 2552.474,  loss: -2205.964,  ess: 4.723


In [ ]:
# torch.save(enc_z.state_dict(), "../weights/enc-z-rad-%s" % PATH)
# torch.save(enc_eta.state_dict(), "../weights/enc-eta-rad-%s" % PATH)
# torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-rad-%s" % PATH)

In [ ]:
from forward_backward_rad import *
MAX_MCMC_STEPS = 24
SAMPLE_SIZE = 10
BATCH_SIZE = 5
Vis_Interval = 6
##
colors = ['r', 'b', 'g']
gs = gridspec.GridSpec(BATCH_SIZE, 2+int(MAX_MCMC_STEPS / Vis_Interval))
gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0, hspace=0)
fig = plt.figure(figsize=(15,15))

indices = torch.arange(NUM_DATASETS)
step = 13
batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
obs = Data[batch_indices]
obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
if CUDA:
    obs =obs.cuda().to(DEVICE)
    
xs = obs[0].cpu().data.numpy()    
for b in range(BATCH_SIZE):
    xb = xs[b]
    ax = fig.add_subplot(gs[b, 0])
    ax.scatter(xb[:, 0], xb[:, 1], c='k')
    ax.set_ylim([-12, 12])
    ax.set_xlim([-12, 12])
    ax.set_xticks([])
    ax.set_yticks([])
    if b == 0:
        ax.set_title('Data', fontsize=30)
        
(oneshot_eta, enc_eta, enc_z) = models
q_eta, p_eta = oneshot_eta(obs, K)
obs_mu = q_eta['means'].value
obs_rad = q_eta['radi'].value
q_z, p_z = enc_z.forward(obs, obs_mu, obs_rad, noise_sigma, 1.0)
state = q_z['zs'].value
E_z = q_z['zs'].dist.probs.mean(0).cpu().data.numpy()
E_mu = q_eta['means'].dist.loc.mean(0).cpu().data.numpy()
for b in range(BATCH_SIZE):
    ax = fig.add_subplot(gs[b, 1])
    xb = xs[b]
    zb = E_z[b]
    mu = E_mu[b].reshape(K, D)
    assignments = zb.argmax(-1)
    for k in range(K):
        xk = xb[np.where(assignments == k)]
        ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
    ax.set_ylim([-12, 12])
    ax.set_xlim([-12, 12])
    ax.set_xticks([])
    ax.set_yticks([])
    if b == 0:
        ax.set_title('Step 0', fontsize=30)
        
for m in range(MAX_MCMC_STEPS):
    q_eta, p_eta = enc_eta(obs, state)
    obs_mu, obs_rad, log_w_eta_f, log_w_eta_b  = Incremental_eta(q_eta, p_eta, obs, state, noise_sigma, obs_mu, obs_rad)
    q_z, p_z = enc_z.forward(obs, obs_mu, obs_rad, noise_sigma, temperature=1.0)
    state, log_w_z_f, log_w_z_b = Incremental_z(q_z, p_z, obs, obs_mu, obs_rad, noise_sigma, state)
    E_z = q_z['zs'].dist.probs.mean(0).cpu().data.numpy()
    E_mu = q_eta['means'].dist.loc.mean(0).cpu().data.numpy()
    if (m+1) % Vis_Interval == 0:
        for b in range(BATCH_SIZE):
            ax = fig.add_subplot(gs[b, 1 + int((m+1)/Vis_Interval)])
            xb = xs[b]
            zb = E_z[b]
            mu = E_mu[b].reshape(K, D)
            assignments = zb.argmax(-1)
            for k in range(K):
                xk = xb[np.where(assignments == k)]
                ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
            ax.set_ylim([-12, 12])
            ax.set_xlim([-12, 12])
            ax.set_xticks([])
            ax.set_yticks([])
            if b == 0:
                ax.set_title('Step %d' % (m+1), fontsize=30)        

In [ ]:
radi = (q_eta['radi'].dist.concentration[0].cpu().data.numpy() / q_eta['radi'].dist.rate[0].cpu().data.numpy())[-1]
mu = q_eta['means'].dist.loc[0].cpu().data.numpy()[-1]
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(xb[:, 0], xb[:, 1], c='k')
ax.set_xlim([-10, 5])
ax.set_ylim([-8, 7])
samples = np.linspace(0, 2*math.pi, 100, endpoint=False)
xcoor = np.cos(samples)
ycoor = np.sin(samples)
for k in range(K):
    xcoork = xcoor * radi[k] + mu[k, 0]
    ycoork = ycoor * radi[k] + mu[k, 1]
    ax.scatter(xcoork, ycoork)

In [ ]:
from ag_ep_rad import *

BATCH_SIZE_TEST = 25
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, 10, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
Model_Params_Test = (noise_sigma, N, K, D, 20)
obs,reused = test(models, EUBO, Data, Model_Params_Test, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples_v2(obs, q_mu, q_z, K, PATH)